# Downloading and Exploring the data

In [9]:
import pandas as pd
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [1]:
#importing and reading the data as dataframe


data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header =0 )
col = ['Postcode', 'Borough','Neighbourhood']
for data in data:
    current_col = data.columns.values[:3]
    if len(current_col) != len(col):
        continue
    if all(current_col == col):
        break
        

In [2]:
data.head(11)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [3]:
#to check the number of columns in dataframe

data.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [4]:
#cleaning the data
data.drop(data.loc[data['Neighbourhood']=='Not assigned'].index, inplace=True)
data.reset_index(drop=True, inplace=True)

data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,Rouge
7,M1B,Scarborough,Malvern
8,M3B,North York,Don Mills North
9,M4B,East York,Woodbine Gardens


In [6]:
# Processing the data
data['Neighbourhood'].replace('Not assigned', data['Borough'], inplace=True)
data.head(11)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,Rouge
7,M1B,Scarborough,Malvern
8,M3B,North York,Don Mills North
9,M4B,East York,Woodbine Gardens


In [7]:
# Creating a new dataframe with merged Neighbourhood
df = pd.DataFrame(data.groupby(['Postcode','Borough'], sort = False)['Neighbourhood'].apply(",".join))
df.reset_index(inplace = True)
df.head(11)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M1B,Scarborough,"Rouge,Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens,Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M6B,North York,Glencairn


In [8]:
# Number of rows and columns
df.shape

(102, 3)

# Generating latitude and longitude for each neighbourhood

In [27]:
file = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df1 = pd.read_csv(file)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [98]:
new_df = df.sort_values(by = 'Postcode')
new_df = pd.DataFrame(new_df)
new_df['Latitude'] = df1['Latitude']
new_df['Longitude'] = df1['Longitude']

new_df.set_index('Postcode',inplace = True)
new_df.reset_index(inplace = True)
new_df.head(11)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.744734,-79.239476
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.750072,-79.295849
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.803762,-79.363452
3,M1G,Scarborough,Woburn,43.789053,-79.408493
4,M1H,Scarborough,Cedarbrae,43.753259,-79.329656
5,M1J,Scarborough,Scarborough Village,43.739015,-79.506944
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.676357,-79.293031
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.659526,-79.340923
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.679563,-79.377529
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.657952,-79.387383
